# how_to_access_the_sensor_data_for_Building_1

This notebook shows how access the sensor data for Building 1.

It creates a csv file for all the Building 1 data, resampled to 30 minute intervals.

## Import packages

In [1]:
import collections
from lxml import etree
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

## Setup data folder

This is the folder where the xml file and csv file are stored. This can be modified as needed.

In [2]:
data_folder=r'..\..\..\_Data\REFIT_Smart_Home_dataset_PUBLIC_v1\Data'

## Read the xml file

Parses the refitXML file into an lxml etree variable

In [3]:
xml_fn=r'REFIT_BUILDING_SURVEY.xml'
tree=etree.parse(os.path.join(data_folder,xml_fn))
NS={'a':'http://www.refitsmarthomes.org'}

## Read the csv file

In [4]:
csv_fn=r'REFIT_TIME_SERIES_VALUES.csv' 
csv=pd.read_csv(os.path.join(data_folder,csv_fn))
csv.head()

TimeSeriesVariable/@id              dateTime    data
0    TimeSeriesVariable1  2013-10-02T05:00:00Z  17.772
1    TimeSeriesVariable1  2013-10-02T05:30:00Z  18.081
2    TimeSeriesVariable1  2013-10-02T06:00:00Z  18.176
3    TimeSeriesVariable1  2013-10-02T06:30:00Z  18.176
4    TimeSeriesVariable1  2013-10-02T07:00:00Z  18.105

## Get Building 1 data

This gets all the data associated with Building 1.

In [5]:
b=tree.xpath('/a:RefitXML/a:Stock/a:Building[@id="Building01"]',namespaces=NS)[0]
tsvs=b.xpath('.//a:TimeSeriesVariable',namespaces=NS)
tsv_ids=[tsv.get('id') for tsv in tsvs]
df=csv[csv['TimeSeriesVariable/@id'].isin(tsv_ids)].copy()
df.head()

TimeSeriesVariable/@id              dateTime    data
0    TimeSeriesVariable1  2013-10-02T05:00:00Z  17.772
1    TimeSeriesVariable1  2013-10-02T05:30:00Z  18.081
2    TimeSeriesVariable1  2013-10-02T06:00:00Z  18.176
3    TimeSeriesVariable1  2013-10-02T06:30:00Z  18.176
4    TimeSeriesVariable1  2013-10-02T07:00:00Z  18.105

## Pivot data

When pivoting the dataframe, any duplicate index/column pairs will result in a ValueError. So duplicates need to be somehow handled. In this case duplicate values are removed. The sensor models where this will occur is given as below. In the case of the RWE sensors, the duplicate values are often legitmate readings so the pivoting process will result in the loss of some useful information here.

In [6]:
x=df[df.duplicated(subset=['dateTime','TimeSeriesVariable/@id'], keep=False)]
unique_tsv_ids=x['TimeSeriesVariable/@id'].unique()
result=[' '.join(tree.xpath('.//a:TimeSeriesVariable[@id="%s"]/../@*[name()="manufacturer" or name()="model"]' % tsv_id,namespaces=NS)) for tsv_id in unique_tsv_ids]
result=set(result)
result

{'Maxim iButton',
 'RWE Alarm',
 'RWE Contact sensor',
 'RWE Exterior motion detector',
 'RWE Interior motion detector',
 'RWE Smart Home Control',
 'RWE Smoke detector',
 'RWE TRV'}

The duplicate values are now dropped, keeping the first values only

In [7]:
df1=df.drop_duplicates(subset=['dateTime','TimeSeriesVariable/@id'])
df1.head()

TimeSeriesVariable/@id              dateTime    data
0    TimeSeriesVariable1  2013-10-02T05:00:00Z  17.772
1    TimeSeriesVariable1  2013-10-02T05:30:00Z  18.081
2    TimeSeriesVariable1  2013-10-02T06:00:00Z  18.176
3    TimeSeriesVariable1  2013-10-02T06:30:00Z  18.176
4    TimeSeriesVariable1  2013-10-02T07:00:00Z  18.105

The dataframe is now pivoted.

(NaN means that data wasn't recorded for a variable at a particular timestamps. In the table below the only data being recorded at the start is for 'TimeSeriesVariable1554')

In [8]:
df2=df1.pivot(index='dateTime',columns='TimeSeriesVariable/@id',values='data')
print(df2.shape)
df2.head()

(214409, 126)


TimeSeriesVariable/@id  TimeSeriesVariable1  TimeSeriesVariable10  \
dateTime                                                            
2013-09-14T00:00:00Z                    NaN                   NaN   
2013-09-14T00:30:00Z                    NaN                   NaN   
2013-09-14T01:00:00Z                    NaN                   NaN   
2013-09-14T01:30:00Z                    NaN                   NaN   
2013-09-14T02:00:00Z                    NaN                   NaN   

TimeSeriesVariable/@id  TimeSeriesVariable11  TimeSeriesVariable12  \
dateTime                                                             
2013-09-14T00:00:00Z                     NaN                   NaN   
2013-09-14T00:30:00Z                     NaN                   NaN   
2013-09-14T01:00:00Z                     NaN                   NaN   
2013-09-14T01:30:00Z                     NaN                   NaN   
2013-09-14T02:00:00Z                     NaN                   NaN   

TimeSeriesVariable/@id  TimeSeriesVariable13  TimeSeriesVariable14  \
dateTime                                                             
2013-09-14T00:00:00Z                     NaN                   NaN   
2013-09-14T00:30:00Z                     NaN                   NaN   
2013-09-14T01:00:00Z                     NaN                   NaN   
2013-09-14T01:30:00Z                     NaN                   NaN   
2013-09-14T02:00:00Z                     NaN                   NaN   

TimeSeriesVariable/@id  TimeSeriesVariable15  TimeSeriesVariable1554  \
dateTime                                                               
2013-09-14T00:00:00Z                     NaN                     0.0   
2013-09-14T00:30:00Z                     NaN                     0.0   
2013-09-14T01:00:00Z                     NaN                     0.0   
2013-09-14T01:30:00Z                     NaN                     0.0   
2013-09-14T02:00:00Z                     NaN                     0.0   

TimeSeriesVariable/@id  TimeSeriesVariable16  TimeSeriesVariable1604  ...  \
dateTime                                                              ...   
2013-09-14T00:00:00Z                     NaN                     NaN  ...   
2013-09-14T00:30:00Z                     NaN                     NaN  ...   
2013-09-14T01:00:00Z                     NaN                     NaN  ...   
2013-09-14T01:30:00Z                     NaN                     NaN  ...   
2013-09-14T02:00:00Z                     NaN                     NaN  ...   

TimeSeriesVariable/@id  TimeSeriesVariable755  TimeSeriesVariable756  \
dateTime                                                               
2013-09-14T00:00:00Z                      NaN                    NaN   
2013-09-14T00:30:00Z                      NaN                    NaN   
2013-09-14T01:00:00Z                      NaN                    NaN   
2013-09-14T01:30:00Z                      NaN                    NaN   
2013-09-14T02:00:00Z                      NaN                    NaN   

TimeSeriesVariable/@id  TimeSeriesVariable757  TimeSeriesVariable758  \
dateTime                                                               
2013-09-14T00:00:00Z                      NaN                    NaN   
2013-09-14T00:30:00Z                      NaN                    NaN   
2013-09-14T01:00:00Z                      NaN                    NaN   
2013-09-14T01:30:00Z                      NaN                    NaN   
2013-09-14T02:00:00Z                      NaN                    NaN   

TimeSeriesVariable/@id  TimeSeriesVariable759  TimeSeriesVariable760  \
dateTime                                                               
2013-09-14T00:00:00Z                      NaN                    NaN   
2013-09-14T00:30:00Z                      NaN                    NaN   
2013-09-14T01:00:00Z                      NaN                    NaN   
2013-09-14T01:30:00Z                      NaN                    NaN   
2013-09-14T02:00:00Z                      NaN  

The index is converted into pd.Timestamp values, and the columns are given more useful names

In [9]:
def tsv_info(tsv):
    sensor=tsv.getparent()
    entity=sensor.getparent()
    return {'TimeSeriesVariable id':tsv.get('id'),
            'variableType':tsv.get('variableType'),
            'units':tsv.get('units'),
            'Sensor manufacturer and model':'%s %s' % (sensor.get('manufacturer'),sensor.get('model')),
            'Entity id':entity.get('id'),
            'roomType':entity.get('roomType')
           }
df3=df2.copy()
df3.index=pd.to_datetime(df3.index)
tsvs=[tree.xpath('.//a:TimeSeriesVariable[@id="%s"]' % tsv_id,namespaces=NS)[0] for tsv_id in df3.columns]
d=collections.defaultdict(list)
for tsv in tsvs:
    for k,v in tsv_info(tsv).items():
        d[k].append(v)
d=dict(d)
arrays=list(d.values())
tuples=list(zip(*arrays))
names=list(d.keys())
df3.columns = pd.MultiIndex.from_tuples(tuples, names=names)
df3.head()

TimeSeriesVariable id         TimeSeriesVariable1 TimeSeriesVariable10  \
variableType                      Air temperature      Air temperature   
units                                           C                    C   
Sensor manufacturer and model      Onset Hobo U12   Onset Hobo pendant   
Entity id                                 Space13              Space17   
roomType                                 Bathroom              Bedroom   
dateTime                                                                 
2013-09-14 00:00:00+00:00                     NaN                  NaN   
2013-09-14 00:30:00+00:00                     NaN                  NaN   
2013-09-14 01:00:00+00:00                     NaN                  NaN   
2013-09-14 01:30:00+00:00                     NaN                  NaN   
2013-09-14 02:00:00+00:00                     NaN                  NaN   

TimeSeriesVariable id         TimeSeriesVariable11 TimeSeriesVariable12  \
variableType                       Air temperature      Air temperature   
units                                            C                    C   
Sensor manufacturer and model   Onset Hobo pendant       Onset Hobo U12   
Entity id                                  Space17              Space17   
roomType                                   Bedroom              Bedroom   
dateTime                                                                  
2013-09-14 00:00:00+00:00                      NaN                  NaN   
2013-09-14 00:30:00+00:00                      NaN                  NaN   
2013-09-14 01:00:00+00:00                      NaN                  NaN   
2013-09-14 01:30:00+00:00                      NaN                  NaN   
2013-09-14 02:00:00+00:00                      NaN                  NaN   

TimeSeriesVariable id         TimeSeriesVariable13 TimeSeriesVariable14  \
variableType                       Air temperature      Air temperature   
units                                            C                    C   
Sensor manufacturer and model   Onset Hobo pendant   Onset Hobo pendant   
Entity id                                  Space14              Space14   
roomType                                   Bedroom              Bedroom   
dateTime                                                                  
2013-09-14 00:00:00+00:00                      NaN                  NaN   
2013-09-14 00:30:00+00:00                      NaN                  NaN   
2013-09-14 01:00:00+00:00                      NaN                  NaN   
2013-09-14 01:30:00+00:00                      NaN                  NaN   
2013-09-14 02:00:00+00:00                      NaN                  NaN   

TimeSeriesVariable id         TimeSeriesVariable15  \
variableType                       Air temperature   
units                                            C   
Sensor manufacturer and model   Onset Hobo pendant   
Entity id                                  Space14   
roomType                                   Bedroom   
dateTime                                             
2013-09-14 00:00:00+00:00                      NaN   
2013-09-14 00:30:00+00:00                      NaN   
2013-09-14 01:00:00+00:00                      NaN   
2013-09-14 01:30:00+00:00                      NaN   
2013-09-14 02:00:00+00:00                      NaN   

TimeSeriesVariable id                                 TimeSeriesVariable1554  \
variableType                                                      Gas volume   
units                                                           Cubic metres   
Sensor manufacturer and model Smart Metering Solutions Replacement gas meter   
Entity id                                                             Meter1   
roomType                                                                 NaN   
dateTime                                                                       
2013-09-14 00:00:00+00:00                                                0.0   
2013-09-14 00:30:00+

The data is then resampled to half hour intervals. For each new interval, the mean of all original values is reported.

In [10]:
df4=df3.resample('30T').mean()
df4.head()

TimeSeriesVariable id         TimeSeriesVariable1 TimeSeriesVariable10  \
variableType                      Air temperature      Air temperature   
units                                           C                    C   
Sensor manufacturer and model      Onset Hobo U12   Onset Hobo pendant   
Entity id                                 Space13              Space17   
roomType                                 Bathroom              Bedroom   
dateTime                                                                 
2013-09-14 00:00:00+00:00                     NaN                  NaN   
2013-09-14 00:30:00+00:00                     NaN                  NaN   
2013-09-14 01:00:00+00:00                     NaN                  NaN   
2013-09-14 01:30:00+00:00                     NaN                  NaN   
2013-09-14 02:00:00+00:00                     NaN                  NaN   

TimeSeriesVariable id         TimeSeriesVariable11 TimeSeriesVariable12  \
variableType                       Air temperature      Air temperature   
units                                            C                    C   
Sensor manufacturer and model   Onset Hobo pendant       Onset Hobo U12   
Entity id                                  Space17              Space17   
roomType                                   Bedroom              Bedroom   
dateTime                                                                  
2013-09-14 00:00:00+00:00                      NaN                  NaN   
2013-09-14 00:30:00+00:00                      NaN                  NaN   
2013-09-14 01:00:00+00:00                      NaN                  NaN   
2013-09-14 01:30:00+00:00                      NaN                  NaN   
2013-09-14 02:00:00+00:00                      NaN                  NaN   

TimeSeriesVariable id         TimeSeriesVariable13 TimeSeriesVariable14  \
variableType                       Air temperature      Air temperature   
units                                            C                    C   
Sensor manufacturer and model   Onset Hobo pendant   Onset Hobo pendant   
Entity id                                  Space14              Space14   
roomType                                   Bedroom              Bedroom   
dateTime                                                                  
2013-09-14 00:00:00+00:00                      NaN                  NaN   
2013-09-14 00:30:00+00:00                      NaN                  NaN   
2013-09-14 01:00:00+00:00                      NaN                  NaN   
2013-09-14 01:30:00+00:00                      NaN                  NaN   
2013-09-14 02:00:00+00:00                      NaN                  NaN   

TimeSeriesVariable id         TimeSeriesVariable15  \
variableType                       Air temperature   
units                                            C   
Sensor manufacturer and model   Onset Hobo pendant   
Entity id                                  Space14   
roomType                                   Bedroom   
dateTime                                             
2013-09-14 00:00:00+00:00                      NaN   
2013-09-14 00:30:00+00:00                      NaN   
2013-09-14 01:00:00+00:00                      NaN   
2013-09-14 01:30:00+00:00                      NaN   
2013-09-14 02:00:00+00:00                      NaN   

TimeSeriesVariable id                                 TimeSeriesVariable1554  \
variableType                                                      Gas volume   
units                                                           Cubic metres   
Sensor manufacturer and model Smart Metering Solutions Replacement gas meter   
Entity id                                                             Meter1   
roomType                                                                 NaN   
dateTime                                                                       
2013-09-14 00:00:00+00:00                                                0.0   
2013-09-14 00:30:00+

## Save results to csv

In [11]:
df4.to_csv('Building_01.csv')